In [3]:
import requests

# Petfinder API credentials
API_KEY = "KONoQXEA5Gf4oVjZT592CS8Ptkxqc6aqTyPgjHa1kVeRAuyWoU"
API_SECRET = "kcmUgm3tQzx7SNlqS59FoTt5NzeXajg7s7oIQLAm"

def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()  # Ensure the request was successful
    return response.json()["access_token"]

def count_adoptable_dogs(city, token):
    """Count the total number of adoptable dogs in a specified city."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "type": "Dog",
        "location": city,
        "limit": 100,  # Adjust as needed, up to the API's max limit
    }
    total_dogs = 0
    page = 1  # Start from the first page

    while True:
        params["page"] = page  # Update the page parameter for each request
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Ensure the request was successful
        data = response.json()
        total_dogs += len(data["animals"])  # Add count of dogs in the current page

        # Check if we've reached the last page
        if page >= data["pagination"]["total_pages"]:
            break  # Exit loop if on the last page
        page += 1  # Move to the next page

    return total_dogs

# Authenticate and get the total number of adoptable dogs in New York City
token = get_auth_token()
total_dogs_nyc = count_adoptable_dogs("New York, NY", token)
print(f"Total number of adoptable dogs in New York City: {total_dogs_nyc}")

Total number of adoptable dogs in New York City: 1087


In [6]:
# This should work in theory but I don't have the API bandwidth to do it

import requests
import pandas as pd

# Petfinder API credentials
API_KEY = "KONoQXEA5Gf4oVjZT592CS8Ptkxqc6aqTyPgjHa1kVeRAuyWoU"
API_SECRET = "kcmUgm3tQzx7SNlqS59FoTt5NzeXajg7s7oIQLAm"

def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()  # Ensure the request was successful
    return response.json()["access_token"]

def fetch_adoptable_pets(cities, token):
    """Fetch adoptable pets (cats and dogs) information for specified cities."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    pets_data = []

    for city in cities:
        for animal_type in ['Cat', 'Dog']:
            params = {
                "type": animal_type,
                "location": city,
                "limit": 100,
            }
            page = 1  # Start from the first page

            while True:
                params["page"] = page
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()

                for animal in data["animals"]:
                    pets_data.append({
                        "type": animal["type"],
                        "breed": animal["breeds"]["primary"],
                        "location": animal["contact"]["address"]["city"],
                        "color": animal["colors"]["primary"] if animal["colors"]["primary"] else "N/A",
                        "size": animal["size"],
                        "date_posted": animal["published_at"],
                        "name": animal["name"],
                        "status": animal["status"],
                        "gender": animal["gender"]
                    })

                if page >= data["pagination"]["total_pages"]:
                    break
                page += 1

    return pd.DataFrame(pets_data)

# Cities to fetch data for
cities = ["New York, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ"]

# Authenticate
token = get_auth_token()

# Fetch pets data
pets_df = fetch_adoptable_pets(cities, token)

# Display the DataFrame
print(pets_df.head())

HTTPError: 429 Client Error: Too Many Requests for url: https://api.petfinder.com/v2/animals?type=Cat&location=New+York%2C+NY&limit=100&page=1